In [3]:

import numpy as np
import pandas as pd
import rebound
import spiceypy as spice
from tqdm import tqdm

import sys
sys.path.append('..')
from geminids.constants import *

def perihelion(age=400):
    spice.furnsh( 'data/meta.tm' )
    # t_start = spice.str2et('1600-01-01')
    beg = spice.str2et('2018 A.D. Jan 1')
    t_start = beg
    year = spice.jyear()
    beg -= age*year
    end = 2*year

    pts_per_year = 365*24
    n_particles = 1
    n_years = 2


    [y0, lt] = spice.spkezr('2003200', t_start, 'ECLIPJ2000', 'NONE', 'SUN')
    y0 = spice.convrt(y0, "KM", "AU")


    n = 1



    sim = rebound.Simulation()


    sim.units = ('s', 'AU', 'Msun')
    sim.dt = -.001
    sim.add(m=1.)

    [yj, lt] = spice.spkezr("JUPITER BARYCENTER", t_start, 
                            "ECLIPJ2000", "NONE", "SUN")
    yj = spice.convrt(yj, "KM", "AU")
    sim.add(m=0.000954588, x=yj[0], y=yj[1], z=yj[2], 
            vx=yj[3], vy=yj[4], vz=yj[5])
    print("Added Jupiter...")

    [e_pos, lt] = spice.spkezr('EARTH', t_start, 'ECLIPJ2000', 'NONE', 'SUN')
    e_pos = spice.convrt(e_pos, 'KM', 'AU')
    sim.add(m=MASS_E/MASS_SUN, x=e_pos[0], y=e_pos[1], z=e_pos[2],
            vx=e_pos[3], vy=e_pos[4], vz=e_pos[5])
    print("Added Earth...")


    [mr_pos, lt] = spice.spkezr('4', t_start, 'ECLIPJ2000', 'NONE', 'SUN')
    mr_pos = spice.convrt(mr_pos, 'KM', 'AU')
    sim.add(m=MASS_MR/MASS_SUN, x=mr_pos[0], y=mr_pos[1], z=mr_pos[2], 
            vx=mr_pos[3], vy=mr_pos[4], vz=mr_pos[5])
    print("Added Mars...")



    [v_pos, lt] = spice.spkezr('VENUS', t_start, 'ECLIPJ2000', 'NONE', 'SUN')
    v_pos = spice.convrt(v_pos, 'KM', 'AU')
    sim.add(m=MASS_V/MASS_SUN, x=v_pos[0], y=v_pos[1], z=v_pos[2], 
            vx=v_pos[3], vy=v_pos[4], vz=v_pos[5])
    print("Added Venus...")



    [hg_pos, lt] = spice.spkezr('MERCURY', t_start, 'ECLIPJ2000', 'NONE', 'SUN')
    hg_pos = spice.convrt(hg_pos, 'KM', 'AU')
    sim.add(m=MASS_HG/MASS_SUN, x=hg_pos[0], y=hg_pos[1], z=hg_pos[2], 
            vx=hg_pos[3], vy=hg_pos[4], vz=hg_pos[5])
    print("Added Mercury...")


    n_active = len(sim.particles)




    sim.n_active = n_active
    sim.collision = "none"



    sim.move_to_hel()
    print("adding particles...")

    sim.add(x = y0[0], y=y0[1], z=y0[2], vx=y0[3], vy = y0[4], vz = y0[5])
    sim.move_to_com()




    print("time arrays...")

    Noutputs = n_years*pts_per_year
    year = spice.jyear()
    times = np.linspace(end-t_start, beg-t_start, int(Noutputs))


    t2 = np.linspace(0, end-t_start, int(1600 + 3000)*100)
    pos = np.zeros((n,int(Noutputs),3))

    sim.move_to_com()        
    ps = sim.particles      



    xy = np.zeros((int(Noutputs),6))
    xy2 = np.zeros((int(1600 + 3000)*100,3))
    d = np.zeros(int(Noutputs))
    print("simulating...")
    p = sim.particles[6]

    for i, time in enumerate(tqdm(t2)):
        sim.integrate(time)
        xy2[i] = [p.x, p.y, p.z]    
    for i, time in enumerate(tqdm(times)):
        sim.integrate(time)
        sim.move_to_hel()
        xy[i] = [p.x, p.y, p.z, p.vx, p.vy, p.vz]
        d[i] = spice.vnorm(xy[i][0:3])
        sim.move_to_com()

    start_date = t_start

    idxmin = np.argmin(d)
    t_start = times[idxmin] + start_date
    perihelion = xy[idxmin]

    return sim


sim_old = perihelion()

  0%|          | 2140/460000 [00:00<00:21, 21395.53it/s]

Added Jupiter...
Added Earth...
Added Mars...
Added Venus...
Added Mercury...
adding particles...
time arrays...
simulating...


100%|██████████| 17520/17520 [00:03<00:00, 4993.13it/s]


In [5]:
sim_old.status()

o_old = sim_old.particles[-1].calculate_orbit()

---------------------------------
REBOUND version:     	3.17.3
REBOUND built on:    	Jun 23 2021 16:00:24
Number of particles: 	7
Selected integrator: 	ias15
Simulation time:     	-1.2623040000000000e+10
Current timestep:    	-163983.554869
---------------------------------
<rebound.particle.Particle object at 0x7fd74d7b13c0, m=1.0 x=-0.0033760055349325814 y=0.0034571175640373693 z=6.117555589302634e-05 vx=-5.947272227977178e-11 vy=-6.165949431047271e-11 vz=1.5952293538287665e-12>
<rebound.particle.Particle object at 0x7fd74d7b1440, m=0.000954588 x=3.538527731603294 y=-3.6172867224059404 z=-0.06415445385802916 vx=6.137017010172056e-08 vy=6.522831857065303e-08 vz=-1.6440972848742294e-09>
<rebound.particle.Particle object at 0x7fd74d7b13c0, m=3.003500178529221e-06 x=-0.1658718474819258 y=-0.9973145926663122 z=-0.0008120431118652975 vx=1.9347232350325574e-07 vy=-3.267143358995608e-08 vz=-7.031701282432449e-12>
<rebound.particle.Particle object at 0x7fd74d7b1440, m=3.2271545459574447e-07 x

In [6]:
age = 400
beg = spice.str2et('1617 A.D. Dec 29')
t_start = beg
year = spice.jyear()
beg -= age*year
end = 2*year

pts_per_year = 365*24*600
n_particles = 1
n_years = 2
    
[y0, lt] = spice.spkezr('2003200', t_start, 'ECLIPJ2000', 'NONE', 'SUN')
y0 = spice.convrt(y0, "KM", "AU")


n = 1



sim = rebound.Simulation()


sim.units = ('s', 'AU', 'Msun')
sim.dt = -.001
sim.add(m=1.)

[yj, lt] = spice.spkezr("JUPITER BARYCENTER", t_start, 
                        "ECLIPJ2000", "NONE", "SUN")
yj = spice.convrt(yj, "KM", "AU")
sim.add(m=0.000954588, x=yj[0], y=yj[1], z=yj[2], 
        vx=yj[3], vy=yj[4], vz=yj[5])
print("Added Jupiter...")

[e_pos, lt] = spice.spkezr('EARTH', t_start, 'ECLIPJ2000', 'NONE', 'SUN')
e_pos = spice.convrt(e_pos, 'KM', 'AU')
sim.add(m=MASS_E/MASS_SUN, x=e_pos[0], y=e_pos[1], z=e_pos[2],
        vx=e_pos[3], vy=e_pos[4], vz=e_pos[5])
print("Added Earth...")


[mr_pos, lt] = spice.spkezr('4', t_start, 'ECLIPJ2000', 'NONE', 'SUN')
mr_pos = spice.convrt(mr_pos, 'KM', 'AU')
sim.add(m=MASS_MR/MASS_SUN, x=mr_pos[0], y=mr_pos[1], z=mr_pos[2], 
        vx=mr_pos[3], vy=mr_pos[4], vz=mr_pos[5])
print("Added Mars...")



[v_pos, lt] = spice.spkezr('VENUS', t_start, 'ECLIPJ2000', 'NONE', 'SUN')
v_pos = spice.convrt(v_pos, 'KM', 'AU')
sim.add(m=MASS_V/MASS_SUN, x=v_pos[0], y=v_pos[1], z=v_pos[2], 
        vx=v_pos[3], vy=v_pos[4], vz=v_pos[5])
print("Added Venus...")



[hg_pos, lt] = spice.spkezr('MERCURY', t_start, 'ECLIPJ2000', 'NONE', 'SUN')
hg_pos = spice.convrt(hg_pos, 'KM', 'AU')
sim.add(m=MASS_HG/MASS_SUN, x=hg_pos[0], y=hg_pos[1], z=hg_pos[2], 
        vx=hg_pos[3], vy=hg_pos[4], vz=hg_pos[5])
print("Added Mercury...")


n_active = len(sim.particles)




sim.n_active = n_active
sim.collision = "none"



sim.move_to_hel()
print("adding particles...")

sim.add(x = y0[0], y=y0[1], z=y0[2], vx=y0[3], vy = y0[4], vz = y0[5])
sim.move_to_com()

Added Jupiter...
Added Earth...
Added Mars...
Added Venus...
Added Mercury...
adding particles...


In [7]:
o = sim.particles[-1].calculate_orbit()

In [14]:
a_err = (o.a - o_old.a)/o.a
e_err = (o.e - o_old.e)/o.e
inc_err = (o.inc - o_old.inc)/o.inc
omega_err = (o.omega - o_old.omega)/o.omega
Omega_err = (o.Omega - o_old.Omega)/o.Omega

print(f"a:{a_err:.2e}, e:{e_err:.2e}, inc:{inc_err:.2e}, omega:{omega_err:.2e}, Omega:{Omega_err:.2e},")

a:-1.11e-03, e:4.18e-04, inc:1.24e-03, omega:-9.27e-04, Omega:-2.30e-03,


In [22]:
print((sim_old.calculate_energy() - sim.calculate_energy()) / sim.calculate_energy())
print((np.array(sim_old.calculate_angular_momentum()) - np.array(sim.calculate_angular_momentum())) / sim.calculate_angular_momentum())


-2.984341084293514e-05
[-7.44371327e-03  6.98709896e-02 -3.84910754e-05]
